In [21]:
import pandas as pd

In [35]:
faresData = pd.read_csv('ticket_fares.csv')
titanicData = pd.read_csv('titanic_cleaned.csv')

faresDf = pd.DataFrame(faresData)
df = pd.DataFrame(titanicData)

df = pd.merge(df, faresDf, on='Ticket', how='left')
df.drop('Fare_x', axis=1, inplace=True)
df.rename(columns={'Fare_y': 'Fare'}, inplace=True)

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Embarked,FamilySize,IsAlone,Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,S,2,0,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,C,2,0,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,S,1,1,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,S,2,0,53.1000
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,S,2,0,53.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,S,1,1,30.0000
1589,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,22,1,2,W./C. 6607,S,4,0,23.4500
1590,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,22,1,2,W./C. 6607,S,4,0,23.4500
1591,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,C,1,1,30.0000


In [ ]:
bins = [0, 13, 20, 39, df['Age'].max() + 1]
labels = ['Child', 'Teen', 'Adult', 'Senior']

df['AgeGroup'] = pd.cut(df['Age'], bins = bins, labels = labels, right = False)
survivalRates = df.groupby(['Sex', 'AgeGroup'])['Survived'].mean().unstack() #unstack learnt from google for how to remove last line of dtype

childSurvivalRate = df[df['AgeGroup'] == 'Child']['Survived'].mean()
womenSurvivalRate = df[(df['Sex'] == 'female') & (df['AgeGroup'] != 'Child')]['Survived'].mean()
menSurvivalRate = df[(df['Sex'] =='male') & (df['AgeGroup'] != 'Child')]['Survived'].mean()

C:\Users\GM A\AppData\Local\Temp\ipykernel_28184\2963934705.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survivalRates = df.groupby(['Sex', 'AgeGroup'])['Survived'].mean().unstack()


In [24]:
with open('report.txt', 'w') as f:
    f.write("Hypothesis 1: Women and Children First\n\n")

    f.write("Survival Analysis by Sex and Age Group:\n")
    f.write(f"\n{survivalRates}\n")
    f.write("Conclusion:\n")
    f.write(
        f"The data overwhelmingly supports the 'women and children first' prioritization. "
        f"Overall, **Children (male and female)** had a survival rate of **{childSurvivalRate:.2%}**. "
        f"**Women (non-child)** had the highest survival rate at **{womenSurvivalRate:.2%}**. "
        f"In stark contrast, **Men (non-child)** had a significantly lower survival rate of **{menSurvivalRate:.2%}**. "
        f"Thus, the data proves a clear order of rescue: **Women, followed by Children, with Adult Men being the lowest priority.**"
    )

In [33]:
survivalRatesbyPclass = df.groupby(['Pclass'])['Survived'].mean()
survivalRatesbyPclass

Pclass
1    0.689055
2    0.480000
3    0.239057
Name: Survived, dtype: float64

In [51]:
df['FareBin'] = pd.qcut(df['Fare'], q=4, labels=['Low', 'Medium', 'High', 'VeryHigh'], precision=2)

survivalRatesByFarebin = df.groupby('FareBin')['Survived'].mean()


highClassRate = survivalRatesbyPclass[1].round(3)
lowClassRate = survivalRatesbyPclass[3].round(3)

highFareRate = survivalRatesByFarebin['VeryHigh'].round(3)
lowFareRate = survivalRatesByFarebin['Low'].round(3)

C:\Users\GM A\AppData\Local\Temp\ipykernel_28184\1038070625.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survivalRatesByFarebin = df.groupby('FareBin')['Survived'].mean()


In [52]:
with open('report.txt', 'a') as f:
    f.write("\n\n---\n\n") # Section separator
    f.write("Hypothesis 2: Wealth\n\n")

    # Write the detailed analysis for reference
    f.write("Method A (Pclass) Survival Rates:\n")
    f.write(survivalRatesbyPclass.apply(lambda x: f'{x:.2%}').to_string() + "\n\n")

    f.write("Method B (FareBin) Survival Rates:\n")
    f.write(survivalRatesByFarebin.apply(lambda x: f'{x:.2%}').to_string() + "\n\n")
    
    # Write the concise concluding paragraph
    f.write("Conclusion (Comparison of Methods A and B):\n")
    f.write(
        f"The data clearly establishes a strong correlation between wealth and survival, demonstrated consistently by both methods. "
        f"Passengers in the **highest wealth bracket** (1st Class and Very High Fare) showed overwhelming advantage: 1st Class passengers survived at a rate of **{highClassRate:.2%}**, and those paying the highest fares survived at **{highFareRate:.2%}**. "
        f"Conversely, passengers in the **lowest wealth bracket** (3rd Class and Low Fare) were the least likely to survive. 3rd Class passengers had a survival rate of only **{lowClassRate:.2%}**, while those in the lowest fare quantile survived at **{lowFareRate:.2%}**. "
        f"Because both Method A (discrete class) and Method B (continuous fare) show identical trends—higher wealth equals higher survival—the hypothesis that **socio-economic status determined access to lifeboats** is strongly supported."
    )